# Bot Testing Grounds
This notebook tests the performance of a series of bots from draftsimtools.

### Package Importing

First, we load relevant packages, including the custom draftsimtools module. 

In [1]:
# Imports packages 

import warnings
warnings.filterwarnings('ignore')

import pickle
import ast
import numpy as np
import datetime  
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data.dataset import Dataset

# Workaround for variable Jupyter directories
import sys
sys.path.append('bots')

import draftsimtools as ds
from draftsimtools import DraftNet

### Data Loading

Next, we set filepaths for raw drafts, the MTG Json file containing detailed info on every card, and for curated draftsim ratings of each card in the current set. In this notebook, we will be only be working with M19 drafts.

To get access to the raw drafts and draftsim rankings, please contact [Dan Troha](https://draftsim.com/contact/).

In [2]:
# Sets pytorch device
device = torch.device("cpu") 

# Sets file paths
# jsonPath = "../../data/AllSets.json"
ratingPath = "bots_data/nnet_train/standardized_m19_rating.tsv"
draftPath = "bots_data/nnet_train/drafts_test.pkl"

# Sets file paths for Bayesian bot
pCollPath = "bots_data/bayes_pCoDraft.csv"
pPackPath = "bots_data/bayes_pChoice.csv"
pFullPath = "bots_data/bayes_pFull.csv"
namesPath = "bots_data/bayes_names.csv"

We read in raw drafts and Draftsim card rankings here. We also create a label encoder object to map packs to binary presence/absence vectors, which is necessary for some bots.

In [3]:
# Loads drafts
drafts = None
with open(draftPath, "rb") as f:
    drafts = pickle.load(f)

# Loads ratings
m19_set = pd.read_csv(ratingPath, delimiter="\t", converters={6:ast.literal_eval})

# Label-encodes card names
le = ds.create_le(m19_set["Name"].values)

For demonstration purposes, we subset the full set of testing drafts (~22k) to just 100 drafts. 

In [4]:
# Subsets drafts for faster runtimes - for real testing, use all drafts
subset_drafts = drafts[:5000] 
print(len(drafts))
if True:
    drafts = drafts[:100]

21590


In [5]:
for col in m19_set:
    print(col)

Name
Casting Cost 1
Casting Cost 2
Card Type
Rarity
Rating
Color Vector


### Testing Bots

We need to instantiate all of the different drafting agents.

**RandomBot**: Picks cards randomly. 

**RaredraftBot**: Picks the rarest cards in its most-dominant color.

**ClassicBot**: Picks cards with the highest draftsim score in its most-dominant colors. 

**BayesBot**: Picks cards based on a Bayesian estimate of how often they co-occur with cards in a collection. 

**NNetBot**: Picks cards based on a deep neural network trained to predict picks given collections. 

In [6]:
# Instantiates heuristic-based bots
bot1 = ds.RandomBot() 
bot2 = ds.RaredraftBot(m19_set) 
bot3 = ds.ClassicBot(m19_set) 
bot4 = ds.BayesBot(le, pCollPath, pPackPath, pFullPath, namesPath)

# Loads neural net from saved pytorch file
test_net = torch.load("bots_data/draftnet_june23_2020.pt")
test_net.eval()

# Instantiates neural-network bot
bot5 = ds.NeuralNetBot(test_net, le)

# Loads neural net from saved pytorch file
test_custom_net = torch.load("bots_data/draftnetCustom_jan31_2022_ep.pt")
test_custom_net.eval()

# Instantiates neural-network bot
bot6 = ds.NeuralNetBot(test_custom_net, le)


                     Name
0      Abnormal_Endurance
1          Act_of_Treason
2    Aegis_of_the_Heavens
3         Aerial_Engineer
4           Aether_Tunnel
..                    ...
260        Walking_Corpse
261          Wall_of_Mist
262         Wall_of_Vines
263     Windreader_Sphinx
264       Woodland_Stream

[265 rows x 1 columns]


Finally, we test all of the different bots against each other by measuring their top-one accuracy on predicting human choices in the subset 100 drafts. The overall accuracy for all bots across all drafts is output, as well as csv files containing bot predictions across all drafts. 

In [7]:
# Tests all bots in the testing framework
tester = ds.BotTester(drafts)
before = datetime.datetime.now()
tester.evaluate_bots([bot1, bot2, bot3, bot4, bot5, bot6], ["RandomBot", "RaredraftBot", "ClassicBot", "BayesBot", "NNetBot", "NNetCustomBot"])
print("Total time taken for", len(drafts) ,"drafts:",datetime.datetime.now() - before)
tester.report_evaluations()
tester.write_evaluations()

Initialization time taken: 0:00:00.007555
RandomBot time taken: 0:00:00.061625
RaredraftBot time taken: 0:00:10.132156
ClassicBot time taken: 0:00:06.428515
BayesBot time taken: 0:00:04.850858
NNetBot time taken: 0:00:04.879359
NNetCustomBot time taken: 0:00:04.854772
Total time taken for 100 drafts: 0:00:32.280130
draft_num        50.500000
pick_num         23.000000
RandomBot         0.226889
RaredraftBot      0.292667
ClassicBot        0.448889
BayesBot          0.431778
NNetBot           0.471556
NNetCustomBot     0.474000
dtype: float64
Wrote correct to: output_files/exact_correct.tsv
Wrote fuzzy_correct to: output_files/fuzzy_correct.tsv
Wrote rank_error to: output_files/rank_error.tsv
Wrote card_acc to: output_files/card_accuracies.tsv


### Rank Example Pack

To illustrate how the bots' interface works, we show how the NNetBot ranks cards in a single pack. 

In [9]:
# Instantiates bot tester
tester = ds.BotTester(drafts)

# Create demo collection
demo_collection = tester.drafts[0][0]
demo_pack = tester.drafts[0][1]
demo_x = ds.collection_pack_to_x(demo_collection, demo_pack, le)

# Return the result
result = test_net(demo_x)

# Maps numeric classes to card names and displays result
pack_dict = {str(le.inverse_transform([i])[0]) : float(v.detach().numpy()) for i, v in enumerate(result[0,:]) if v > 0}
display(pack_dict)

Initialization time taken: 0:00:00.007351


{'Boggart_Brute': 17.24131965637207,
 'Bristling_Boar': 16.391267776489258,
 'Disperse': 12.952291488647461,
 'Duress': 12.309479713439941,
 'Dwarven_Priest': 15.90805435180664,
 'Ghirapur_Guide': 16.10085678100586,
 'Macabre_Waltz': 12.962005615234375,
 'Regal_Bloodlord': 15.658961296081543,
 'Revitalize': 14.009382247924805,
 "Rogue's_Gloves": 15.011547088623047,
 'Salvager_of_Secrets': 12.661223411560059,
 'Submerged_Boneyard': 12.239510536193848,
 'Viashino_Pyromancer': 16.456260681152344,
 'Walking_Corpse': 12.505791664123535}